In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

### read provided url, add to beautifulsoup, find the class that contains the table and get all the rows

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')



### All all the rows into a list

In [3]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)

### create df

In [4]:
df = pd.DataFrame(l, columns=["PostCode", "Borough", "Neighbourhood"])

### remove borough cell that has value 'not assigned' 

In [5]:
df = df[(df.Borough != 'Not assigned')]

### remove '\n' from the string

In [6]:
df['Neighbourhood'] = df['Neighbourhood'].str.replace('\n', '').str.strip()

### join the neighbourhoods together based on same postal code

In [7]:
df = df.groupby(['PostCode','Borough'], as_index=False, sort=True).agg(', '.join)

### assigned the borough value to neighbourhoods that have a 'not assigned' value

In [8]:
for index, row in df.iterrows():
    if(row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough'] 
df.shape    

(103, 3)